# 02 WIP: Development

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
os.chdir("..")
os.getcwd()

In [ ]:
from src.gridgpt.template_manager import (
    load_templates,
    select_template,
    print_template_grid,
    identify_theme_slots
)

In [ ]:
template = select_template(difficulty="easy")

In [ ]:
print_template_grid(template)

In [ ]:
identify_theme_slots(template)

# Crossword Generator

In [ ]:
from src.gridgpt.crossword_generator import CrosswordGenerator
from src.gridgpt.crossword_generator import generate_themed_crossword

In [ ]:
generator = CrosswordGenerator()

In [ ]:
theme_entry = "BREWS"

In [ ]:
templates_data = load_templates()
template = select_template(templates_data, difficulty='easy')
print_template_grid(template)

In [ ]:
generator.validate_theme_entry(theme_entry)

In [ ]:
template_with_theme = generator.place_theme_entry(template, theme_entry)
template_with_theme

In [ ]:
crossword = generate_themed_crossword(template, "SUITS")

In [ ]:
crossword

In [ ]:
#!/usr/bin/env python3
import argparse
import json
import logging
import os
import sys

from src.gridgpt.template_manager import load_templates, select_template, print_template_grid
from src.gridgpt.crossword_generator import generate_themed_crossword

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def parse_args():
    """Parse command-line arguments."""
    parser = argparse.ArgumentParser(description='Generate a crossword puzzle')
    
    parser.add_argument(
        '--template', 
        help='ID of template to use (default: random)'
    )
    
    parser.add_argument(
        '--theme-entry',
        help='Theme entry to include in the crossword'
    )
    
    parser.add_argument(
        '--difficulty',
        choices=['easy', 'medium', 'hard'],
        help='Difficulty level of the crossword'
    )
    
    parser.add_argument(
        '--output',
        help='Output file to save the crossword (JSON format)'
    )
    
    parser.add_argument(
        '--verbose', 
        '-v', 
        action='store_true',
        help='Enable verbose logging'
    )
    
    return parser.parse_args()

def main():
    """Main function to generate a crossword puzzle."""
    # Parse command-line arguments
    args = parse_args()
    
    # Set logging level
    if args.verbose:
        logging.getLogger().setLevel(logging.DEBUG)
    
    # Load templates
    try:
        templates_data = load_templates()
    except Exception as e:
        logger.error(f"Failed to load templates: {e}")
        sys.exit(1)
    
    # Select template
    try:
        template = select_template(templates_data, template_id=args.template, difficulty=args.difficulty)
    except Exception as e:
        logger.error(f"Failed to select template: {e}")
        sys.exit(1)
    
    # Print the selected template
    print_template_grid(template)
    
    # Generate crossword
    try:
        crossword = generate_themed_crossword(template, args.theme_entry)
    except ValueError as e:
        logger.error(f"Failed to generate crossword: {e}")
        sys.exit(1)
    
    # Save to output file if specified
    if args.output:
        try:
            with open(args.output, 'w', encoding='utf-8') as f:
                json.dump(crossword, f, indent=2)
            logger.info(f"Crossword saved to {args.output}")
        except Exception as e:
            logger.error(f"Failed to save crossword: {e}")
    
    return 0

if __name__ == "__main__":
    sys.exit(main())

# Generate Clues

In [ ]:
from src.gridgpt.clue_generator import generate_mixed_clues

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

In [ ]:
theme = "general knowledge"
clues = generate_mixed_clues(crossword, theme)

In [ ]:
# Print clues
print("\nClues:")
print("\nAcross:")
across_clues = {k: v for k, v in clues.items() if 'A' in k}
for slot_id, clue in sorted(across_clues.items()):
    word = crossword["filled_slots"][slot_id]
    print(f"{slot_id}: {clue} ({word})")
    
print("\nDown:")
down_clues = {k: v for k, v in clues.items() if 'D' in k}
for slot_id, clue in sorted(down_clues.items()):
    word = crossword["filled_slots"][slot_id]
    print(f"{slot_id}: {clue} ({word})")

In [ ]:
crossword['grid']

In [ ]:
# TODO: add nyt sample clues, add clue character limits and add nyt clue types (abbr, language, quotes, etc)